In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import pandas as pd
import json
train_data = "/content/drive/MyDrive/Subtask_1_train.json"
f = open(train_data)
data = json.load(f)
f.close()

In [3]:
data[0]['emotion_utterance_ID']

'dia1utt3'

In [4]:
d = pd.read_json(train_data)

In [5]:
d

,emotion_utterance_ID,emotion,conversation,cause_spans
0,dia1utt3,surprise,"[{'utterance_ID': 1, 'text': 'Alright , so I a...","[1_I realize I am totally naked ., 3_Then I lo..."
1,dia1utt4,surprise,"[{'utterance_ID': 1, 'text': 'Alright , so I a...","[1_I realize I am totally naked ., 3_Then I lo..."
2,dia1utt5,anger,"[{'utterance_ID': 1, 'text': 'Alright , so I a...","[1_I realize I am totally naked ., 3_Then I lo..."
3,dia2utt1,sadness,"[{'utterance_ID': 1, 'text': 'I do not want to...",[1_I do not want to be single]
4,dia2utt3,surprise,"[{'utterance_ID': 1, 'text': 'I do not want to...",[3_Rachel ? !]
...,...,...,...,...
6756,dia1360utt11,anger,"[{'utterance_ID': 1, 'text': 'Wow ! Fortunatel...",[10_keep having sex on a regular basis .]
6757,dia1371utt1,joy,"[{'utterance_ID': 1, 'text': 'Ooh ! I am sorry...",[1_that may have missed the table !]
6758,dia1371utt6,anger,"[{'utterance_ID': 1, 'text': 'Ooh ! I am sorry...",[1_that may have missed the table !]
6759,dia1374utt2,surprise,"[{'utterance_ID': 1, 'text': 'I never sucked ,...","[1_I never sucked , I actually did not want yo..."


In [6]:
tagging_data = d
x = tagging_data["emotion_utterance_ID"].str.extract(r"dia([\d]+)utt([\d]+)", expand=False)
tagging_data["dialogue_ID"], tagging_data["emotion_ID"] = x[0].astype(int), x[1].astype(int)
tagging_data["dialogue_ID"], tagging_data["emotion_ID"]

(0          1
 1          1
 2          1
 3          2
 4          2
         ... 
 6756    1360
 6757    1371
 6758    1371
 6759    1374
 6760    1374
 Name: dialogue_ID, Length: 6761, dtype: int64,
 0        3
 1        4
 2        5
 3        1
 4        3
         ..
 6756    11
 6757     1
 6758     6
 6759     2
 6760     4
 Name: emotion_ID, Length: 6761, dtype: int64)

In [7]:
tagging_data["enc_cause_spans"] = tagging_data["cause_spans"].apply(lambda x: [int(y[0]) for y in x])
tagging_data["enc_cause_spans"]

0          [1, 3]
1       [1, 3, 4]
2       [1, 3, 4]
3             [1]
4             [3]
          ...    
6756          [1]
6757          [1]
6758          [1]
6759          [1]
6760          [4]
Name: enc_cause_spans, Length: 6761, dtype: object

In [8]:
tagging_data["trunc_conversation"] = tagging_data.apply(lambda x: [y for y in x["conversation"] if y["utterance_ID"] <= x["emotion_ID"]], axis=1)
tagging_data["trunc_conversation"]

0       [{'utterance_ID': 1, 'text': 'Alright , so I a...
1       [{'utterance_ID': 1, 'text': 'Alright , so I a...
2       [{'utterance_ID': 1, 'text': 'Alright , so I a...
3       [{'utterance_ID': 1, 'text': 'I do not want to...
4       [{'utterance_ID': 1, 'text': 'I do not want to...
                              ...                        
6756    [{'utterance_ID': 1, 'text': 'Wow ! Fortunatel...
6757    [{'utterance_ID': 1, 'text': 'Ooh ! I am sorry...
6758    [{'utterance_ID': 1, 'text': 'Ooh ! I am sorry...
6759    [{'utterance_ID': 1, 'text': 'I never sucked ,...
6760    [{'utterance_ID': 1, 'text': 'I never sucked ,...
Name: trunc_conversation, Length: 6761, dtype: object

In [9]:
#sanity check
tagging_data["trunc_conversation"].iloc[0], tagging_data["trunc_conversation"].iloc[1]

([{'utterance_ID': 1,
   'text': 'Alright , so I am back in high school , I am standing in the middle of the cafeteria , and I realize I am totally naked .',
   'speaker': 'Chandler',
   'emotion': 'neutral'},
  {'utterance_ID': 2,
   'text': 'Oh , yeah . Had that dream .',
   'speaker': 'All',
   'emotion': 'neutral'},
  {'utterance_ID': 3,
   'text': 'Then I look down , and I realize there is a phone ... there .',
   'speaker': 'Chandler',
   'emotion': 'surprise'}],
 [{'utterance_ID': 1,
   'text': 'Alright , so I am back in high school , I am standing in the middle of the cafeteria , and I realize I am totally naked .',
   'speaker': 'Chandler',
   'emotion': 'neutral'},
  {'utterance_ID': 2,
   'text': 'Oh , yeah . Had that dream .',
   'speaker': 'All',
   'emotion': 'neutral'},
  {'utterance_ID': 3,
   'text': 'Then I look down , and I realize there is a phone ... there .',
   'speaker': 'Chandler',
   'emotion': 'surprise'},
  {'utterance_ID': 4,
   'text': 'Instead of ... ?',


In [10]:
tagging_data.drop(labels=["conversation", "emotion_utterance_ID", "cause_spans"], axis=1, inplace=True)

In [11]:
tagging_data.rename(columns={"emotion_ID": "utterance_ID", "enc_cause_spans": "cause_span", "trunc_conversation":"t_conv"}, inplace=True)
tagging_data

,emotion,dialogue_ID,utterance_ID,cause_span,t_conv
0,surprise,1,3,"[1, 3]","[{'utterance_ID': 1, 'text': 'Alright , so I a..."
1,surprise,1,4,"[1, 3, 4]","[{'utterance_ID': 1, 'text': 'Alright , so I a..."
2,anger,1,5,"[1, 3, 4]","[{'utterance_ID': 1, 'text': 'Alright , so I a..."
3,sadness,2,1,[1],"[{'utterance_ID': 1, 'text': 'I do not want to..."
4,surprise,2,3,[3],"[{'utterance_ID': 1, 'text': 'I do not want to..."
...,...,...,...,...,...
6756,anger,1360,11,[1],"[{'utterance_ID': 1, 'text': 'Wow ! Fortunatel..."
6757,joy,1371,1,[1],"[{'utterance_ID': 1, 'text': 'Ooh ! I am sorry..."
6758,anger,1371,6,[1],"[{'utterance_ID': 1, 'text': 'Ooh ! I am sorry..."
6759,surprise,1374,2,[1],"[{'utterance_ID': 1, 'text': 'I never sucked ,..."


### Find a way to encode the conversation data

encoding must include,
aggregation of word embeddings
just concatenate the embeddings? to sentence embeddings?
speaker embeddings
emotion embeddings

In [66]:
tagging_data["concat_conv"] = tagging_data["t_conv"].apply(lambda x: [ {"utterance_ID": y["utterance_ID"], "data":y["text"]+" "+y["speaker"]+" "+y["emotion"]} for y in x])

In [14]:
import pickle
with open("/content/drive/MyDrive/tagging_data.pkl", 'wb') as f:
  pickle.dump(tagging_data, f)

## Embedding with Doc2Vec

In [67]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec

In [110]:
all_utterances = tagging_data.apply(lambda x: [{"data":y["data"], "id":str(x["dialogue_ID"])+"utt"+str(y["utterance_ID"])} for y in x["concat_conv"]], axis=1).tolist()
utt = []
for x in all_utterances:
  utt.extend(x)

In [93]:
embedding_size = 128

In [112]:
documents = [TaggedDocument(d["data"].split(), [d["id"]]) for d in utt]
model = Doc2Vec(documents, vector_size=embedding_size, window=3, min_count=1, workers=4)

In [115]:
import pickle
with open("/content/drive/MyDrive/doc2vec_concat_128.pkl", 'wb') as f:
  pickle.dump(model, f)

## Word2Vec and Doc2Vec

In [12]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [33]:
from gensim.models import Word2Vec


In [56]:
all_utterances = tagging_data.apply(lambda x: [y["text"].split() for y in x["t_conv"]], axis=1).tolist()
utt = []
for x in all_utterances:
  utt.extend(x)

In [57]:
all_authors = tagging_data.apply(lambda x: [y["speaker"].split() for y in x["t_conv"]], axis=1).tolist()
auth = []
for x in all_authors:
  auth.extend(x)

In [58]:
all_emotions = tagging_data.apply(lambda x: [y["emotion"].split() for y in x["t_conv"]], axis=1).tolist()
emot = []
for x in all_emotions:
  emot.extend(x)

In [93]:
all_ids =  tagging_data.apply(lambda x: [str(x["dialogue_ID"])+"utt"+str(y["utterance_ID"]) for y in x["t_conv"]], axis=1).tolist()
ids = []
for x in all_ids:
  ids.extend(x)

In [59]:
model = Word2Vec(sentences=auth+emot+utt, vector_size=128, window=5, min_count=1, workers=4)

In [62]:
#sanity check
for a in auth:
  try:
    model.wv[a]
  except KeyError:
    break

In [55]:
#sanity check
for index, word in enumerate(model.wv.index_to_key):
    if index == 10:
        break
    print(f"word #{index}/{len(wv.index_to_key)} is {word}")

word #0/3000000 is Ross
word #1/3000000 is Joey
word #2/3000000 is Rachel
word #3/3000000 is Monica
word #4/3000000 is Phoebe
word #5/3000000 is Chandler
word #6/3000000 is The
word #7/3000000 is Janice
word #8/3000000 is Dr.
word #9/3000000 is Mr.


In [95]:
all_author_embeds = [model.wv[a] for a in auth]
all_emot_embeds = [model.wv[e] for e in emot]

In [96]:
all_utterances = tagging_data.apply(lambda x: [{"data":y["data"], "id":str(x["dialogue_ID"])+"utt"+str(y["utterance_ID"])} for y in x["concat_conv"]], axis=1).tolist()
utt = []
for x in all_utterances:
  utt.extend(x)

In [97]:
documents = [TaggedDocument(d["data"].split(), [d["id"]]) for d in utt]
model = Doc2Vec(documents, vector_size=128, window=3, min_count=1, workers=4)

In [98]:
all_utt_embeds = []
for u in utt:
  all_utt_embeds.append(model.dv[u["id"]])

In [99]:
import numpy as np
import pickle


In [104]:
added_embeds = {}
maxed_embeds = {}
avged_embeds = {}
for u, a, e, id in zip(all_utt_embeds, all_author_embeds, all_emot_embeds, ids):
  a, e = a[0], e[0]
  added_embeds[id] = u+a+e
  maxed_embeds[id] = np.maximum.reduce([u,a,e])
  avged_embeds[id] = (u+a+e)/128

In [105]:
with open("/content/drive/MyDrive/word2vec_add_128.pkl", 'wb') as f:
  pickle.dump(added_embeds, f)

In [106]:
with open("/content/drive/MyDrive/word2vec_max_128.pkl", 'wb') as f:
  pickle.dump(maxed_embeds, f)

In [107]:
with open("/content/drive/MyDrive/word2vec_avg_128.pkl", 'wb') as f:
  pickle.dump(avged_embeds, f)